# Content-based Filtering

Aproach:
- Table with features erstellen, ratings in zweitem table

- sklearn-kNN mit minkowski-distance auf features anwenden, erhalte matrix mit allen Nachbarn eines jeden Films inklusive Abstand

- Wähle für jeden Nutzer und Film die x-nächsten Nachbarn aus, die der Nutzer bewertet hat

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
import time
kwargs = dict(random_state=42)

In [2]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')
actors = pd.read_csv('../../data/raw/actors.csv', sep=',')

In [3]:
mapping = movies[['id', 'imdbID']].rename(columns={'id':'movieID'})

## Feature Preprocessing

In [4]:
movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [5]:
ratings

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0
...,...,...,...,...
787536,812812,1032,tt0083530,3.0
787537,812813,99,tt0107798,3.0
787538,812814,333,tt0093857,3.0
787539,812815,49,tt0144168,3.0


In [6]:
omdb

,0,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,...,Series,Released_season,Released_month,Released_day,PG_Rating,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated,Available_languages
0,tt0047034,Godzilla,1954.0,Not Rated,96.0,"Takeo Murata, Ishirô Honda, Shigeru Kayama (st...",American nuclear weapons testing results in th...,['Japanese'],7.6,27485.0,...,0.0,2.0,5.0,7.0,NaN,0.0,0.0,0.0,0.0,1
1,tt0304141,Harry Potter and the Prisoner of Azkaban,2004.0,PG,142.0,"J.K. Rowling (novel), Steve Kloves (screenplay)","Harry Potter, Ron and Hermione return to Hogwa...",['English'],7.9,524511.0,...,0.0,3.0,6.0,4.0,1.0,0.0,2.0,0.0,0.0,1
2,tt0369436,Four Christmases,2008.0,PG-13,88.0,"Matt Allen (screenplay), Caleb Wilson (screenp...",A couple struggles to visit all four of their ...,['English'],5.7,61335.0,...,0.0,4.0,11.0,26.0,2.0,0.0,0.0,0.0,0.0,1
3,tt1077258,Planet Terror,2007.0,Not Rated,105.0,Robert Rodriguez,"After an experimental bio-weapon is released, ...",['English'],7.1,196149.0,...,0.0,3.0,6.0,13.0,NaN,0.0,0.0,0.0,0.0,1
4,tt0052182,Separate Tables,1958.0,Approved,100.0,"Terence Rattigan (screenplay), John Gay (scree...",The stories of several people are told as they...,['English'],7.4,6337.0,...,0.0,1.0,2.0,11.0,NaN,2.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9302,tt0266860,Pokemon 3: The Movie,2001.0,NaN,87.0,Satoshi Tajiri,"In the town of Greenfield, a young, lonely gir...",['Japanese'],5.9,12672.0,...,0.0,2.0,6.0,21.0,NaN,0.0,0.0,0.0,0.0,1
9303,tt0313487,Pokemon 4Ever,2002.0,NaN,75.0,Hideki Sonoda,Ash must stop a hunter who forces the mythical...,['Japanese'],5.7,7600.0,...,0.0,2.0,7.0,7.0,NaN,0.0,0.0,0.0,0.0,1
9304,tt0383974,Le chagrin et la pitie,1969.0,NaN,251.0,Marcel Ophuls,An in-depth exploration of the various reactio...,['French'],8.2,3484.0,...,0.0,2.0,9.0,18.0,NaN,0.0,0.0,0.0,0.0,1
9305,tt0134948,Just the Ticket,1999.0,NaN,115.0,Richard Wenk,A ticket scalper wants to make one last big de...,['English'],5.7,1822.0,...,0.0,1.0,2.0,26.0,NaN,0.0,0.0,0.0,0.0,1


In [7]:
omdb.columns

Index(['0', 'Title', 'Year', 'Rated', 'Runtime', 'Writer', 'Plot', 'Language',
       'imdbRating', 'imdbVotes', 'imdbID', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_season', 'Released_month', 'Released_day',
       'PG_Rating', 'Oscars_won', 'Oscars_nominated', 'Golden_globe_won',
       'Golden_globe_nominated', 'Available_languages'],
      dtype='object')

In [8]:
ratings

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0
...,...,...,...,...
787536,812812,1032,tt0083530,3.0
787537,812813,99,tt0107798,3.0
787538,812814,333,tt0093857,3.0
787539,812815,49,tt0144168,3.0


In [9]:
 # Fill NaN Runtime
omdb.loc[:, 'Runtime'] = omdb.loc[:, 'Runtime'].fillna(omdb.loc[:, 'Runtime'].median())
# Fill NaN for imdbVotes
omdb.loc[:, 'imdbVotes'] = omdb.loc[:, 'imdbVotes'].fillna(omdb.loc[:, 'imdbVotes'].median())

In [10]:
#Combine awards in one column
omdb['Awards'] = omdb['Oscars_won'] + omdb['Golden_globe_won'] + omdb['Oscars_nominated'] + omdb['Golden_globe_nominated']
omdb = omdb.drop(columns={'Oscars_won', 'Oscars_nominated', 'Golden_globe_won', 'Golden_globe_nominated'})

In [11]:
# how often 
threshold=20

In [12]:
actor_counts = pd.DataFrame(actors['actorID'].value_counts())
actors_selected = actor_counts[actor_counts['actorID']>threshold]
actors_selected = actors.set_index('actorID').loc[actors_selected.index].reset_index()
# merge with imdbID, groupby imdbID and write the x most prominent actors as one entry per movie
actors_grouped = actors_selected.merge(mapping, on='movieID').groupby('imdbID')['actorID'].apply(list).reset_index(name='actors')
mlb = MultiLabelBinarizer()
actors_enc = pd.DataFrame(mlb.fit_transform(actors_grouped['actors']))
actors_grouped = actors_grouped.join(actors_enc)
actors_features = actors_grouped.drop(columns={'actors'})

In [13]:
omdb

,0,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,imdbID,Rotten Tomatoes,Metacritic,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Awards
0,tt0047034,Godzilla,1954.0,Not Rated,96.0,"Takeo Murata, Ishirô Honda, Shigeru Kayama (st...",American nuclear weapons testing results in th...,['Japanese'],7.6,27485.0,tt0047034,9.3,7.8,0.0,2.0,5.0,7.0,NaN,1,0.0
1,tt0304141,Harry Potter and the Prisoner of Azkaban,2004.0,PG,142.0,"J.K. Rowling (novel), Steve Kloves (screenplay)","Harry Potter, Ron and Hermione return to Hogwa...",['English'],7.9,524511.0,tt0304141,9.0,8.2,0.0,3.0,6.0,4.0,1.0,1,2.0
2,tt0369436,Four Christmases,2008.0,PG-13,88.0,"Matt Allen (screenplay), Caleb Wilson (screenp...",A couple struggles to visit all four of their ...,['English'],5.7,61335.0,tt0369436,2.4,4.1,0.0,4.0,11.0,26.0,2.0,1,0.0
3,tt1077258,Planet Terror,2007.0,Not Rated,105.0,Robert Rodriguez,"After an experimental bio-weapon is released, ...",['English'],7.1,196149.0,tt1077258,7.4,NaN,0.0,3.0,6.0,13.0,NaN,1,0.0
4,tt0052182,Separate Tables,1958.0,Approved,100.0,"Terence Rattigan (screenplay), John Gay (scree...",The stories of several people are told as they...,['English'],7.4,6337.0,tt0052182,6.9,NaN,0.0,1.0,2.0,11.0,NaN,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9302,tt0266860,Pokemon 3: The Movie,2001.0,NaN,87.0,Satoshi Tajiri,"In the town of Greenfield, a young, lonely gir...",['Japanese'],5.9,12672.0,tt0266860,NaN,NaN,0.0,2.0,6.0,21.0,NaN,1,0.0
9303,tt0313487,Pokemon 4Ever,2002.0,NaN,75.0,Hideki Sonoda,Ash must stop a hunter who forces the mythical...,['Japanese'],5.7,7600.0,tt0313487,NaN,NaN,0.0,2.0,7.0,7.0,NaN,1,0.0
9304,tt0383974,Le chagrin et la pitie,1969.0,NaN,251.0,Marcel Ophuls,An in-depth exploration of the various reactio...,['French'],8.2,3484.0,tt0383974,NaN,NaN,0.0,2.0,9.0,18.0,NaN,1,0.0
9305,tt0134948,Just the Ticket,1999.0,NaN,115.0,Richard Wenk,A ticket scalper wants to make one last big de...,['English'],5.7,1822.0,tt0134948,NaN,NaN,0.0,1.0,2.0,26.0,NaN,1,0.0


In [14]:
actors_features

,imdbID,0,1,2,3,4,5,6,7,8,...,824,825,826,827,828,829,830,831,832,833
0,tt0004972,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0006864,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt0009968,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0011841,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0012532,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6508,tt1179891,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6509,tt1185834,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6510,tt1190617,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6511,tt1205489,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
def features(threshold_actors=20, ts_languages=10, year=True, runtime=True, imdbVotes=True, series=True, awards=True, genres=True):
    features = omdb[['imdbID']]
    names_features = []
    movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
    mapping = movies[['id', 'imdbID']].rename(columns={'id':'movieID'})
    if year == True:
        features = features.merge(omdb[['Year', 'imdbID']], on='imdbID', how='left')
        names_features.append('Year')
    if runtime == True:
        features = features.merge(omdb[['Runtime', 'imdbID']], on='imdbID', how='left')
        names_features.append('Runtime')
    if imdbVotes == True:
        features = features.merge(omdb[['imdbVotes', 'imdbID']], on='imdbID', how='left')
        names_features.append('imdbVotes')
    if series == True:
        features = features.merge(omdb[['Series', 'imdbID']], on='imdbID', how='left')
        names_features.append('Series')
    if awards == True:
        features = features.merge(omdb[['Awards', 'imdbID']], on='imdbID', how='left')
        names_features.append('Awards')
    if threshold_actors != 0:
        # how often 
        actor_counts = pd.DataFrame(actors['actorID'].value_counts())
        actors_selected = actor_counts[actor_counts['actorID']>threshold_actors]
        actors_selected = actors.set_index('actorID').loc[actors_selected.index].reset_index()
        # merge with imdbID, groupby imdbID and write the x most prominent actors as one entry per movie
        actors_grouped = actors_selected.merge(mapping, on='movieID').groupby('imdbID')['actorID'].apply(list).reset_index(name='actors')
        mlb = MultiLabelBinarizer()
        actors_enc = pd.DataFrame(mlb.fit_transform(actors_grouped['actors']))
        actors_grouped = actors_grouped.join(actors_enc)
        actors_features = actors_grouped.drop(columns={'actors'})
        features = features.merge(actors_features, on='imdbID', how='left')
        names_features.append('Actors'+str(threshold_actors))
    if genres == True: 
        genres = pd.read_csv('../../data/raw/genres.csv', sep=',')
        genres_grouped = genres.merge(mapping, on='movieID').groupby('imdbID')['genre'].apply(list).reset_index(name='genres')
        mlb = MultiLabelBinarizer()
        genres_encoded = mlb.fit_transform(genres_grouped['genres'])
        genres_grouped = genres_grouped.join(pd.DataFrame(genres_encoded))
        genres_grouped = genres_grouped.sort_values('imdbID').drop(columns={'genres'})
        features = features.merge(genres_grouped, on='imdbID', how='left')
        names_features.append('Genres')
    if ts_languages != 0: 
        mlb = MultiLabelBinarizer()
        lg = pd.DataFrame(mlb.fit_transform(omdb['Language']))
        lg_index = pd.DataFrame(lg.sum()>ts_languages)
        language = omdb[['0']].join(lg[lg_index[lg_index[0]].index]).rename(columns={'0':'imdbID'})
        print(language.head(0))
        features = features.merge(language, on='imdbID', how='left')
        names_features.append('Language'+str(ts_languages))
    features = features.fillna(0)
    return features, names_features

In [16]:
features, names = features()

Empty DataFrame
Columns: [imdbID, 0, 1, 4, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 29, 30, 31, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 62, 63]
Index: []

[0 rows x 52 columns]


In [17]:
features

,imdbID,Year,Runtime,imdbVotes,Series,Awards,0_x,1_x,2_x,3_x,...,52_y,53_y,55_y,56_y,57_y,58_y,59_y,60_y,62_y,63_y
0,tt0047034,1954.0,96.0,27485.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,0,0,0,0
1,tt0304141,2004.0,142.0,524511.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
2,tt0369436,2008.0,88.0,61335.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
3,tt1077258,2007.0,105.0,196149.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
4,tt0052182,1958.0,100.0,6337.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9302,tt0266860,2001.0,87.0,12672.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,0,0,0,0
9303,tt0313487,2002.0,75.0,7600.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,0,0,0,0
9304,tt0383974,1969.0,251.0,3484.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
9305,tt0134948,1999.0,115.0,1822.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


## KNN

In [18]:
# Standardize the features
scaler = preprocessing.StandardScaler()
features = scaler.fit_transform(features.drop(columns = {'imdbID'}))

In [19]:
features

array([[-1.8480496 , -0.37457098, -0.15497182, ..., -0.1584705 ,
        -0.05686655, -0.06569925],
       [ 0.86352638,  1.523969  ,  4.02362335, ..., -0.1584705 ,
        -0.05686655, -0.06569925],
       [ 1.08045246, -0.70475185,  0.12961178, ..., -0.1584705 ,
        -0.05686655, -0.06569925],
       ...,
       [-1.0345768 ,  6.0226833 , -0.35675294, ..., -0.1584705 ,
        -0.05686655, -0.06569925],
       [ 0.59236879,  0.40960857, -0.3707257 , ..., -0.1584705 ,
        -0.05686655, -0.06569925],
       [ 0.32121119, -0.20948055, -0.27881823, ..., -0.1584705 ,
        -0.05686655, -0.06569925]])

In [20]:
# features of the first entry (imdbID 439)
features[0]

array([-1.84804960e+00, -3.74570982e-01, -1.54971817e-01, -8.12246733e-02,
       -3.94940069e-01, -5.08465447e-02, -5.49323816e-02, -6.14394488e-02,
       -4.75549038e-02, -4.97732904e-02, -4.75549038e-02, -5.18978328e-02,
       -5.29284636e-02, -5.39396214e-02, -5.18978328e-02, -6.81294612e-02,
       -4.27776186e-02, -4.52288658e-02, -4.64063355e-02, -4.64063355e-02,
       -6.14394488e-02, -4.52288658e-02, -5.18978328e-02, -6.05521166e-02,
       -4.64063355e-02, -4.52288658e-02, -4.86766169e-02, -5.18978328e-02,
       -5.08465447e-02, -4.86766169e-02, -5.49323816e-02, -4.75549038e-02,
       -5.59077243e-02, -5.29284636e-02, -4.97732904e-02, -4.52288658e-02,
       -4.52288658e-02, -4.27776186e-02, -4.97732904e-02, -5.18978328e-02,
       -4.64063355e-02, -4.75549038e-02, -5.49323816e-02, -6.14394488e-02,
       -4.52288658e-02, -5.78096682e-02, -4.86766169e-02, -4.75549038e-02,
       -4.52288658e-02, -5.59077243e-02, -6.56992504e-02, -4.75549038e-02,
       -5.49323816e-02, -

In [21]:
features.shape[0]

9307

In [22]:
# create NearesNeighbor model with the minkowski distance
knn = NearestNeighbors(metric='minkowski', p=2, algorithm='brute', n_jobs=-1)

In [23]:
# fit to all the features
knn.fit(features)

NearestNeighbors(algorithm='brute', n_jobs=-1)

In [24]:
# create two array for distance and index to store
distances = np.empty((features.shape[0],features.shape[0]))
indices = np.empty((features.shape[0],features.shape[0]))

# iterate over all imdbIDs
for i in range(0, features.shape[0]):
    # get all neighbors and the distances for each imdbID i
    distances[i], indices[i] = knn.kneighbors([features[i]], n_neighbors=features.shape[0])
    
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [25]:
distances

array([[3.37174788e-07, 6.04877618e+00, 6.18689150e+00, ...,
        9.18082505e+01, 9.93641203e+01, 1.06152147e+02],
       [3.37174788e-07, 5.21595684e+00, 5.36806378e+00, ...,
        8.96170406e+01, 9.69977478e+01, 1.03746032e+02],
       [0.00000000e+00, 4.62350971e+01, 4.78177537e+01, ...,
        1.03357422e+02, 1.09667498e+02, 1.15780990e+02],
       ...,
       [0.00000000e+00, 7.35334003e+00, 7.79380330e+00, ...,
        9.02212797e+01, 9.76409583e+01, 1.04290135e+02],
       [0.00000000e+00, 1.95511663e+01, 2.01378866e+01, ...,
        9.31667349e+01, 1.00143540e+02, 1.03356722e+02],
       [0.00000000e+00, 2.89167567e+00, 3.91444397e+00, ...,
        8.98319075e+01, 9.73583264e+01, 1.04195729e+02]])

In [26]:
indices

array([[0.000e+00, 3.909e+03, 8.932e+03, ..., 5.670e+02, 7.539e+03,
        2.300e+02],
       [1.000e+00, 2.563e+03, 3.115e+03, ..., 5.670e+02, 7.539e+03,
        2.300e+02],
       [2.000e+00, 2.519e+03, 1.296e+03, ..., 5.670e+02, 7.539e+03,
        2.300e+02],
       ...,
       [9.304e+03, 8.364e+03, 5.640e+03, ..., 5.670e+02, 7.539e+03,
        2.300e+02],
       [9.305e+03, 8.280e+02, 2.444e+03, ..., 5.670e+02, 7.539e+03,
        2.300e+02],
       [9.306e+03, 8.482e+03, 9.283e+03, ..., 5.670e+02, 7.539e+03,
        2.300e+02]])

In [27]:
len(indices)

9307

In [28]:
distances_df = pd.DataFrame(distances)

In [29]:
indices_df = pd.DataFrame(indices)

In [30]:
omdb = omdb.reset_index()
omdb

,index,0,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,...,imdbID,Rotten Tomatoes,Metacritic,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Awards
0,0,tt0047034,Godzilla,1954.0,Not Rated,96.0,"Takeo Murata, Ishirô Honda, Shigeru Kayama (st...",American nuclear weapons testing results in th...,['Japanese'],7.6,...,tt0047034,9.3,7.8,0.0,2.0,5.0,7.0,NaN,1,0.0
1,1,tt0304141,Harry Potter and the Prisoner of Azkaban,2004.0,PG,142.0,"J.K. Rowling (novel), Steve Kloves (screenplay)","Harry Potter, Ron and Hermione return to Hogwa...",['English'],7.9,...,tt0304141,9.0,8.2,0.0,3.0,6.0,4.0,1.0,1,2.0
2,2,tt0369436,Four Christmases,2008.0,PG-13,88.0,"Matt Allen (screenplay), Caleb Wilson (screenp...",A couple struggles to visit all four of their ...,['English'],5.7,...,tt0369436,2.4,4.1,0.0,4.0,11.0,26.0,2.0,1,0.0
3,3,tt1077258,Planet Terror,2007.0,Not Rated,105.0,Robert Rodriguez,"After an experimental bio-weapon is released, ...",['English'],7.1,...,tt1077258,7.4,NaN,0.0,3.0,6.0,13.0,NaN,1,0.0
4,4,tt0052182,Separate Tables,1958.0,Approved,100.0,"Terence Rattigan (screenplay), John Gay (scree...",The stories of several people are told as they...,['English'],7.4,...,tt0052182,6.9,NaN,0.0,1.0,2.0,11.0,NaN,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9302,9302,tt0266860,Pokemon 3: The Movie,2001.0,NaN,87.0,Satoshi Tajiri,"In the town of Greenfield, a young, lonely gir...",['Japanese'],5.9,...,tt0266860,NaN,NaN,0.0,2.0,6.0,21.0,NaN,1,0.0
9303,9303,tt0313487,Pokemon 4Ever,2002.0,NaN,75.0,Hideki Sonoda,Ash must stop a hunter who forces the mythical...,['Japanese'],5.7,...,tt0313487,NaN,NaN,0.0,2.0,7.0,7.0,NaN,1,0.0
9304,9304,tt0383974,Le chagrin et la pitie,1969.0,NaN,251.0,Marcel Ophuls,An in-depth exploration of the various reactio...,['French'],8.2,...,tt0383974,NaN,NaN,0.0,2.0,9.0,18.0,NaN,1,0.0
9305,9305,tt0134948,Just the Ticket,1999.0,NaN,115.0,Richard Wenk,A ticket scalper wants to make one last big de...,['English'],5.7,...,tt0134948,NaN,NaN,0.0,1.0,2.0,26.0,NaN,1,0.0


In [31]:
omdb_index = omdb[['index', '0']]
omdb_index

,index,0
0,0,tt0047034
1,1,tt0304141
2,2,tt0369436
3,3,tt1077258
4,4,tt0052182
...,...,...
9302,9302,tt0266860
9303,9303,tt0313487
9304,9304,tt0383974
9305,9305,tt0134948


In [ ]:
ratings = ratings.merge(omdb_index, left_on= 'imdbID', right_on='0', how='left').drop(['0'], axis=1)

In [ ]:
#delete ratings of movie "nomads"
ratings = ratings[ratings['imdbID']!='tt0720339']
ratings

In [ ]:
# First Approach for easy k tuning - use method later to implement in depth tuning of k

def adjust_k(r_size):
    adjusted_k = 10
    if r_size > 40 and r_size  < 100:
        adjusted_k = 15
    elif r_size  > 100 and r_size < 500:
        adjusted_k = 20
    elif r_size  > 500 and r_size < 1500:
        adjusted_k = 25
    elif r_size  > 1500:
        adjusted_k = 30
        #print(r_size) 
        
    return adjusted_k

In [ ]:
#Alles in eine Funktion
def predict_rating(imdbID, userID, mean=True):
    zeitu = time.perf_counter()
    
    #Bestimme Index in omdb von der ImdbID
    index = omdb.loc[(omdb['imdbID'] == imdbID)].index
    
    #Speichere Reihenfolge der Indizes der Nachbarn des Films und tausche erste Reihe mit Spaltenbezeichnungen
    movie_row = indices_df.loc[index]
    movie_row = movie_row.reset_index(drop=True)
    movie_row.columns = movie_row.loc[0]
    movie_row.drop(0, inplace=True)
    movie_row.loc[1,:] = range(0,len(indices_df))
    
    #Speichere alle Filme, die der User bewertet hat und bestimme index davon
    rated_id = ratings.loc[ratings['user_id'] == userID]
    rated_df = rated_id['index']
    rated_df = pd.DataFrame(rated_df)
    
    # Bestimme die Position der bewerteten Filme innerhalb der Nachbarschaft
    position = movie_row[list(rated_df['index'].astype(int))]
    position = position.transpose()
    position.columns = ['position']
    position ['index'] = position.index
    position = position.reset_index(drop = True)
    
    #Greife auf den Film in der Distanzmatrix zu
    movie_row_distance = distances_df.loc[index]
    
    #Bestimme Distanz der Filme vom Film, die der User bewertet hat
    user_distances = movie_row_distance[list(position['position'].astype(int))].transpose()
    user_distances.columns = ['distance']
    user_distances =user_distances.reset_index(drop = True)
    
    #Füge alle Infos zusammen und sortiere von nächstem zum weitest entfernten Film
    neighbors = pd.concat([position, user_distances], axis=1, join = "inner")
    neighbors = neighbors.sort_values("distance")
        
    #Füge die Ratings des Users zu diesen Filmen noch hinzu
    neighbors = neighbors.merge(rated_id[["rating","index"]], on="index", how = "outer")
    neighbors = neighbors.iloc[1:]
    
    #Berechne die Prognose: 
    if mean==True:
        # Mittelwer der k-nächsten Nachbarn
        k = adjust_k(neighbors.shape[0])
        pred = neighbors['rating'].iloc[:k].mean()
    else:
        # Gewichtung der Bewertung der Nachbarn je nach Distanz
        pred = sum(neighbors['rating']*((1/neighbors['distance'])**10)/(sum((1/neighbors['distance'])**10)))
    
    return pred

In [ ]:
n = 100
test_set = ratings.sample(n)
test_set['rating'].isna().sum()
test_set

In [ ]:
zeitu = time.perf_counter()

predictions = np.empty(n)
reals = np.empty(n)
i = 0

for index, row in test_set.iterrows():
        imdbID = row['imdbID']
        userID = row['user_id']
        real = row['rating']
        pred = predict_rating(imdbID, userID)
        predictions[i] = pred
        reals[i] = real
        i = i+1
rmse = mean_squared_error(reals, predictions, squared=False)
results = pd.DataFrame()
results['Features'] = [names]
results['Sample_size'] = n
results['RMSE'] = rmse
results.to_csv('Ergebnisse_Carmen_ContentBased.csv', mode='a', header=False)
print(results)
print('RMSE: '+str(rmse))
zeite = time.perf_counter() - zeitu
print('Gesamtzeit: '+str(zeite))

In [ ]:
test = pd.DataFrame(predictions)
test

In [33]:
#delete ratings of movie "nomads"
ratings = ratings[ratings['imdbID']!='tt0720339']
ratings

,Unnamed: 0,user_id,imdbID,rating,index
0,0,1264,tt0047034,3.5,0.0
1,1,213,tt0304141,2.5,1.0
2,2,593,tt0369436,3.0,2.0
3,3,609,tt1077258,4.0,3.0
4,4,1590,tt0052182,4.0,4.0
...,...,...,...,...,...
787536,812812,1032,tt0083530,3.0,1929.0
787537,812813,99,tt0107798,3.0,1097.0
787538,812814,333,tt0093857,3.0,2215.0
787539,812815,49,tt0144168,3.0,665.0


In [87]:
# First Approach for easy k tuning - use method later to implement in depth tuning of k

def adjust_k(r_size):
    adjusted_k = 10
    if r_size > 40 and r_size  < 100:
        adjusted_k = 15
    elif r_size  > 100 and r_size < 500:
        adjusted_k = 20
    elif r_size  > 500 and r_size < 1500:
        adjusted_k = 25
    elif r_size  > 1500:
        adjusted_k = 30
        #print(r_size) 
        
    return adjusted_k

In [88]:
#Alles in eine Funktion
def predict_rating(imdbID, userID, mean=True):
    zeitu = time.perf_counter()
    
    #Bestimme Index in omdb von der ImdbID
    index = omdb.loc[(omdb['imdbID'] == imdbID)].index
    
    #Speichere Reihenfolge der Indizes der Nachbarn des Films und tausche erste Reihe mit Spaltenbezeichnungen
    movie_row = indices_df.loc[index]
    movie_row = movie_row.reset_index(drop=True)
    movie_row.columns = movie_row.loc[0]
    movie_row.drop(0, inplace=True)
    movie_row.loc[1,:] = range(0,len(indices_df))
    
    #Speichere alle Filme, die der User bewertet hat und bestimme index davon
    rated_id = ratings.loc[ratings['user_id'] == userID]
    rated_df = rated_id['index']
    rated_df = pd.DataFrame(rated_df)
    
    # Bestimme die Position der bewerteten Filme innerhalb der Nachbarschaft
    position = movie_row[list(rated_df['index'].astype(int))]
    position = position.transpose()
    position.columns = ['position']
    position ['index'] = position.index
    position = position.reset_index(drop = True)
    
    #Greife auf den Film in der Distanzmatrix zu
    movie_row_distance = distances_df.loc[index]
    
    #Bestimme Distanz der Filme vom Film, die der User bewertet hat
    user_distances = movie_row_distance[list(position['position'].astype(int))].transpose()
    user_distances.columns = ['distance']
    user_distances =user_distances.reset_index(drop = True)
    
    #Füge alle Infos zusammen und sortiere von nächstem zum weitest entfernten Film
    neighbors = pd.concat([position, user_distances], axis=1, join = "inner")
    neighbors = neighbors.sort_values("distance")
        
    #Füge die Ratings des Users zu diesen Filmen noch hinzu
    neighbors = neighbors.merge(rated_id[["rating","index"]], on="index", how = "outer")
    neighbors = neighbors.iloc[1:]
    
    #Berechne die Prognose: 
    if mean==True:
        # Mittelwer der k-nächsten Nachbarn
        k = adjust_k(neighbors.shape[0])
        pred = neighbors['rating'].iloc[:k].mean()
    else:
        # Gewichtung der Bewertung der Nachbarn je nach Distanz
        pred = sum(neighbors['rating']*((1/neighbors['distance'])**10)/(sum((1/neighbors['distance'])**10)))
    
    return pred

In [45]:
n = 100
test_set = ratings.sample(n)
test_set['rating'].isna().sum()
test_set

,Unnamed: 0,user_id,imdbID,rating,index
137028,140769,865,tt0116282,5.0,1250.0
151348,155474,862,tt0068646,5.0,543.0
490804,505513,17,tt0390022,3.5,4726.0
555164,572009,441,tt0105265,3.0,2370.0
225283,231621,478,tt0104431,0.5,1831.0
...,...,...,...,...,...
591237,609336,513,tt0298148,4.5,842.0
719970,742674,492,tt0032551,4.0,3358.0
60933,62651,670,tt0280105,4.0,6484.0
116368,119550,119,tt0059017,4.5,4535.0


In [89]:
zeitu = time.perf_counter()

predictions = np.empty(n)
reals = np.empty(n)
i = 0

for index, row in test_set.iterrows():
        imdbID = row['imdbID']
        userID = row['user_id']
        real = row['rating']
        pred = predict_rating(imdbID, userID)
        predictions[i] = pred
        reals[i] = real
        i = i+1
rmse = mean_squared_error(reals, predictions, squared=False)
results = pd.DataFrame()
results['Features'] = [names]
results['Sample_size'] = n
results['RMSE'] = rmse
results.to_csv('Ergebnisse_Carmen_ContentBased.csv', mode='a', header=False)
print(results)
print('RMSE: '+str(rmse))
zeite = time.perf_counter() - zeitu
print('Gesamtzeit: '+str(zeite))

2582
1516
1876
2288
1722
1553
1516
2468
1518
1518
                                            Features  Sample_size      RMSE
0  [Year, Runtime, imdbVotes, Series, Awards, Act...          100  0.964074
RMSE: 0.9640738071099928
Gesamtzeit: 2.9797459000001254


In [40]:
test = pd.DataFrame(predictions)
test

,0
0,3.174094
1,3.002756
2,3.704158
3,3.824129
4,3.558203
...,...
95,3.364024
96,3.336388
97,2.978307
98,3.830753
